<a href="https://colab.research.google.com/github/MadmanMarble/MadmanMarble/blob/main/multi_class_U_Net_Model_(11_march_2024).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
# multi-class u-net model

from keras.models import Model
from keras.layers import Input, Conv2D, MaxPooling2D, UpSampling2D, concatenate, Conv2DTranspose, BatchNormalization, Dropout, Lambda
from keras.utils import normalize
import os
import glob
import cv2
import numpy as np
from matplotlib import pyplot as plt

In [3]:
pip install rasterio


In [4]:
import rasterio

In [5]:
def multi_unet_model(n_classes=12, IMG_HEIGHT=128, IMG_WIDTH=128, IMG_CHANNELS=13):
#Build the model
    inputs = Input((IMG_HEIGHT, IMG_WIDTH, IMG_CHANNELS))
    #s = Lambda(lambda x: x / 255)(inputs)   #No need for this if we normalize our inputs beforehand
    s = inputs

    #Contraction path
    c1 = Conv2D(16, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(s)
    c1 = Dropout(0.1)(c1)
    c1 = Conv2D(16, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c1)
    p1 = MaxPooling2D((2, 2))(c1)

    c2 = Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(p1)
    c2 = Dropout(0.1)(c2)
    c2 = Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c2)
    p2 = MaxPooling2D((2, 2))(c2)

    c3 = Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(p2)
    c3 = Dropout(0.2)(c3)
    c3 = Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c3)
    p3 = MaxPooling2D((2, 2))(c3)

    c4 = Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(p3)
    c4 = Dropout(0.2)(c4)
    c4 = Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c4)
    p4 = MaxPooling2D(pool_size=(2, 2))(c4)

    c5 = Conv2D(256, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(p4)
    c5 = Dropout(0.3)(c5)
    c5 = Conv2D(256, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c5)

    #Expansive path
    u6 = Conv2DTranspose(128, (2, 2), strides=(2, 2), padding='same')(c5)
    u6 = concatenate([u6, c4])
    c6 = Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(u6)
    c6 = Dropout(0.2)(c6)
    c6 = Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c6)

    u7 = Conv2DTranspose(64, (2, 2), strides=(2, 2), padding='same')(c6)
    u7 = concatenate([u7, c3])
    c7 = Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(u7)
    c7 = Dropout(0.2)(c7)
    c7 = Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c7)

    u8 = Conv2DTranspose(32, (2, 2), strides=(2, 2), padding='same')(c7)
    u8 = concatenate([u8, c2])
    c8 = Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(u8)
    c8 = Dropout(0.1)(c8)
    c8 = Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c8)

    u9 = Conv2DTranspose(16, (2, 2), strides=(2, 2), padding='same')(c8)
    u9 = concatenate([u9, c1], axis=3)
    c9 = Conv2D(16, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(u9)
    c9 = Dropout(0.1)(c9)
    c9 = Conv2D(16, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c9)

    outputs = Conv2D(n_classes, (1, 1), activation='softmax')(c9)

    model = Model(inputs=[inputs], outputs=[outputs])

    #NOTE: Compile the model in the main program to make it easy to test with various loss functions
    #model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

    #model.summary()

    return model

In [6]:
#Resizing images, if needed
SIZE_X = 128
SIZE_Y = 128
n_classes=12 #Number of classes for segmentation

In [7]:
train_images = sorted(glob.glob(f"/content/drive/MyDrive/DeepLearning_Project/unet_planet_data/planet_images/*.tif"))
train_masks = sorted(glob.glob(f"/content/drive/MyDrive/DeepLearning_Project/unet_planet_data/planet_labels/*.tif"))


In [8]:
# Load the train_images array from the file
train_images = np.load('/content/drive/MyDrive/DeepLearning_Project/unet_planet_data/train_images.npy')

# Verify the shape to ensure the data is loaded correctly
print(train_images.shape)

# Load the train_images array from the file
train_masks = np.load('/content/drive/MyDrive/DeepLearning_Project/unet_planet_data/train_masks.npy')

# Verify the shape to ensure the data is loaded correctly
print(train_masks.shape)

(2825, 13, 384, 384)
(2825, 1, 384, 384)


In [9]:
# # prompt: read train_images as numpy array. each image has 13 bands/channels

# import numpy as np
# # Define the path to the training images directory
# train_images_path = "/content/drive/MyDrive/DeepLearning_Project/unet_planet_data/planet_images"

# # Get a list of all the training image files
# train_images_files = glob.glob(os.path.join(train_images_path, "*.tif"))

# # Initialize an empty list to store the training images as numpy arrays
# train_images = []

# # Loop through each training image file
# for image_file in train_images_files:
#     # Open the image file with rasterio
#     with rasterio.open(image_file) as image:
#         # Read the image data as a numpy array
#         image_data = image.read()

#         # Append the image data to the list
#         train_images.append(image_data)

# # Convert the list of numpy arrays to a single numpy array
# train_images = np.array(train_images)

# # Each image has 13 bands/channels
# print(train_images.shape)


In [10]:
print(train_images.shape)

(2825, 13, 384, 384)


In [11]:
train_images

array([[[[ 1.54000000e+02,  1.54000000e+02,  1.54000000e+02, ...,
           1.52000000e+02,  1.52000000e+02,  1.52000000e+02],
         [ 1.54000000e+02,  1.54000000e+02,  1.54000000e+02, ...,
           1.52000000e+02,  1.52000000e+02,  1.52000000e+02],
         [ 1.54000000e+02,  1.54000000e+02,  1.54000000e+02, ...,
           1.52000000e+02,  1.52000000e+02,  1.52000000e+02],
         ...,
         [ 2.50000000e+02,  2.50000000e+02,  2.50000000e+02, ...,
           4.28000000e+02,  4.28000000e+02,  4.28000000e+02],
         [ 2.50000000e+02,  2.50000000e+02,  2.50000000e+02, ...,
           4.28000000e+02,  4.28000000e+02,  4.28000000e+02],
         [ 2.50000000e+02,  2.50000000e+02,  2.50000000e+02, ...,
           4.28000000e+02,  4.28000000e+02,  4.28000000e+02]],

        [[ 1.05000000e+02,  1.05000000e+02,  1.05000000e+02, ...,
           8.60000000e+01,  8.60000000e+01,  8.60000000e+01],
         [ 1.05000000e+02,  1.05000000e+02,  1.05000000e+02, ...,
           8.60000000e

In [12]:
# # prompt: read train_masks as a numpy array

# import rasterio
# import numpy as np

# # Define the path to the training masks directory
# train_masks_path = "/content/drive/MyDrive/DeepLearning_Project/unet_planet_data/planet_labels"

# # Get a list of all the training mask files
# train_masks_files = glob.glob(os.path.join(train_masks_path, "*.tif"))

# # Initialize an empty list to store the training masks as numpy arrays
# train_masks = []

# # Loop through each training mask file
# for mask_file in train_masks_files:
#     # Open the mask file with rasterio
#     with rasterio.open(mask_file) as mask:
#         # Read the mask data as a numpy array
#         mask_data = mask.read()

#         # Append the mask data to the list
#         train_masks.append(mask_data)

# # Convert the list of numpy arrays to a single numpy array
# train_masks = np.array(train_masks)

# print(train_masks.shape)

In [13]:
np.unique(train_masks)

array([ 0.,  1.,  2.,  3.,  4.,  5.,  6.,  7.,  8.,  9., 10., 11., 12.],
      dtype=float32)

In [14]:
train_masks

array([[[[10., 10., 10., ...,  3.,  3.,  3.],
         [10., 10., 10., ...,  3.,  3.,  3.],
         [10., 10., 10., ...,  3.,  3.,  3.],
         ...,
         [ 6.,  6.,  6., ...,  6.,  6.,  6.],
         [ 6.,  6.,  6., ...,  6.,  6.,  6.],
         [ 6.,  6.,  6., ...,  6.,  6.,  6.]]],


       [[[ 1.,  1.,  1., ...,  2.,  2.,  2.],
         [ 1.,  1.,  1., ...,  2.,  2.,  2.],
         [ 1.,  1.,  1., ...,  2.,  2.,  2.],
         ...,
         [ 6.,  6.,  6., ...,  6.,  6.,  6.],
         [ 6.,  6.,  6., ...,  6.,  6.,  6.],
         [ 6.,  6.,  6., ...,  6.,  6.,  6.]]],


       [[[ 3.,  3.,  3., ...,  3.,  3.,  3.],
         [ 3.,  3.,  3., ...,  3.,  3.,  3.],
         [ 3.,  3.,  3., ...,  3.,  3.,  3.],
         ...,
         [ 3.,  3.,  3., ...,  2.,  2.,  2.],
         [ 3.,  3.,  3., ...,  2.,  2.,  2.],
         [ 1.,  1.,  1., ...,  2.,  2.,  2.]]],


       ...,


       [[[ 7.,  7.,  7., ...,  7.,  7.,  7.],
         [ 7.,  7.,  7., ...,  7.,  7.,  7.],
         [ 7

In [15]:
# Save the train_images array to a file
# np.save('/content/drive/MyDrive/DeepLearning_Project/unet_planet_data/train_images.npy', train_images)
# np.save('/content/drive/MyDrive/DeepLearning_Project/unet_planet_data/train_masks.npy', train_masks)

In [16]:
from sklearn.model_selection import train_test_split

In [17]:
#Create a subset of data for quick testing
#Picking 10% for testing and remaining for training
X1, X_test, y1, y_test = train_test_split(train_images, train_masks, test_size = 0.10, random_state = 0)

In [18]:
#Further split training data t0 a smaller subset for quick testing of models
# X_train, X_do_not_use, y_train, y_do_not_use = train_test_split(X1, y1, test_size = 0.2, random_state = 0)

In [20]:
print("class values in the dataset ", np.unique(y_test))

class values in the dataset  [ 0.  1.  2.  3.  4.  5.  6.  7.  8.  9. 10. 11. 12.]


In [21]:
print(np.unique(y1))

[ 0.  1.  2.  3.  4.  5.  6.  7.  8.  9. 10. 11. 12.]


In [29]:
# prompt: print how many files are in y_test
print('Number of files in y1:', len(y1))
print('Number of files in y_test:', len(y_test))
y1.shape

Number of files in y1: 2542
Number of files in y_test: 283


(2542, 1, 384, 384)

In [30]:
print('Number of files in X1:', len(X1))
print('Number of files in X_test:', len(X_test))
X1.shape

Number of files in X1: 2542
Number of files in X_test: 283


(2542, 13, 384, 384)

In [28]:
from keras.utils import to_categorical

In [ ]:
train_masks_cat = to_categorical(y1, num_classes=n_classes)
y_train_cat = train_masks_cat.reshape((y1.shape[0], y1.shape[1], y1.shape[2], n_classes))

In [ ]:
test_masks_cat = to_categorical(y_test, num_classes=n_classes)
y_test_cat = test_masks_cat.reshape((y_test.shape[0], y_test.shape[1], y_test.shape[2], n_classes))